In [1]:
import warnings
warnings.filterwarnings("ignore")
import sys, os
sys.path.append(os.path.abspath("../.."))
from configs import GOOGLE_APPLICATION_CREDENTIALS,GCS_BUCKET_NAME,GCS_PROJECT_ID
from google.cloud import bigquery
from src.utils.io_utils import upload_to_bigquery
from clean_utils import *

In [2]:
client = bigquery.Client.from_service_account_json(GOOGLE_APPLICATION_CREDENTIALS)
table_id = f"{GCS_PROJECT_ID}.{GCS_BUCKET_NAME}.data_cleaned"

In [3]:
query = """SELECT *
FROM `khangtestdbt.xecupredict.chotot_com` """
data_chotot = client.query(query).to_dataframe()
data_chotot.head(1)

,name,price,Số_Km_đã_đi,Xuất_xứ,Tình_trạng,Chính_sách_bảo_hành,Hãng,Dòng_xe,Năm_sản_xuất,Hộp_số,Nhiên_liệu,Kiểu_dáng,Số_chỗ,Trọng_lượng,Trọng_tải,Số_đời_chủ,Có_phụ_kiện_đi_kèm,Còn_hạn_đăng_kiểm
0,Alphard nhập khẩu NHẬT 2025 - xe giao sớm,4.510.000.000 đ,None,Nhật Bản,Mới,Bảo hành hãng,Toyota,Alphard,2025,Tự động,Động cơ Hybrid,Minivan (MPV),7,None,None,None,None,None


In [4]:
df_chotot = data_chotot.copy()

In [5]:
df_chotot.columns

Index(['name', 'price', 'Số_Km_đã_đi', 'Xuất_xứ', 'Tình_trạng',
       'Chính_sách_bảo_hành', 'Hãng', 'Dòng_xe', 'Năm_sản_xuất', 'Hộp_số',
       'Nhiên_liệu', 'Kiểu_dáng', 'Số_chỗ', 'Trọng_lượng', 'Trọng_tải',
       'Số_đời_chủ', 'Có_phụ_kiện_đi_kèm', 'Còn_hạn_đăng_kiểm'],
      dtype='object')

In [6]:
df_chotot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2632 entries, 0 to 2631
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   name                 2632 non-null   object
 1   price                2632 non-null   object
 2   Số_Km_đã_đi          2361 non-null   object
 3   Xuất_xứ              2632 non-null   object
 4   Tình_trạng           2632 non-null   object
 5   Chính_sách_bảo_hành  2632 non-null   object
 6   Hãng                 2632 non-null   object
 7   Dòng_xe              2632 non-null   object
 8   Năm_sản_xuất         2632 non-null   object
 9   Hộp_số               2632 non-null   object
 10  Nhiên_liệu           2632 non-null   object
 11  Kiểu_dáng            2342 non-null   object
 12  Số_chỗ               2435 non-null   object
 13  Trọng_lượng          2361 non-null   object
 14  Trọng_tải            2361 non-null   object
 15  Số_đời_chủ           1114 non-null   object
 16  Có_phụ

In [7]:
df_chotot = df_chotot[["Số_Km_đã_đi", "Tình_trạng", "Xuất_xứ", "Kiểu_dáng", "Nhiên_liệu", "name", "price", "Hãng", "Năm_sản_xuất"]].copy()

In [8]:
df_chotot.columns = ["km", "Tình_trạng", "origin", "body", "fuel", "name", "price", "brand", "age"]

In [9]:
df_chotot.describe(include='all').T

,count,unique,top,freq
km,2361,523,100000,78
Tình_trạng,2632,2,Đã sử dụng,2361
origin,2632,11,Việt Nam,1149
body,2342,9,SUV / Cross over,894
fuel,2632,4,Xăng,1988
name,2632,2610,VinFast VF5 Plus 2025,3
price,2632,730,295.000.000 đ,26
brand,2632,46,Toyota,561
age,2632,34,2025,313


In [10]:
df_chotot.isna().sum()

km            271
Tình_trạng      0
origin          0
body          290
fuel            0
name            0
price           0
brand           0
age             0
dtype: int64

1.name

In [11]:
df_chotot["name"] = df_chotot["name"].apply(clean_name)
df_chotot.head(1)

,km,Tình_trạng,origin,body,fuel,name,price,brand,age
0,None,Mới,Nhật Bản,Minivan (MPV),Động cơ Hybrid,Alphard nhập khẩu NHẬT 2025 xe giao sớm,4.510.000.000 đ,Toyota,2025


2.price

In [12]:
df_chotot["price"].sample(5)

1834      525.000.000 đ
2362      195.000.000 đ
906       719.000.000 đ
916       575.000.000 đ
1255    1.325.000.000 đ
Name: price, dtype: object

In [13]:
df_chotot["price"] = (
    df_chotot["price"]
    .str.replace("đ", "", regex=False)
    .str.replace(".", "", regex=False)
    .str.strip()
    .astype(float)
    .astype("Int64")
)
df_chotot.price.sample(5)

558    660000000
384    205000000
234    460000000
240    568000000
309    164000000
Name: price, dtype: Int64

In [14]:
df_chotot["price"].isna().sum()

np.int64(0)

In [15]:
df_chotot = df_chotot[df_chotot["price"] > 50000000]

3.brand

In [16]:
df_chotot["brand"].sample(5)

1226    Mitsubishi
1771        Toyota
1178    Mitsubishi
680         Toyota
202         Toyota
Name: brand, dtype: object

In [17]:
df_chotot["brand"].isna().sum()

np.int64(0)

In [18]:
df_chotot["brand"].unique()

array(['Toyota', 'Kia', 'VinFast', 'Hyundai', 'Suzuki', 'Mazda', 'Audi',
       'Honda', 'Mitsubishi', 'Chevrolet', 'Mercedes Benz', 'Ford', 'MG',
       'Daewoo', 'Nissan', 'Lexus', 'BMW', 'Hãng khác', 'Dongfeng',
       'Isuzu', 'Asia', 'SYM', 'Daihatsu', 'Volkswagen', 'Dodge', 'MINI',
       'Baic', 'Peugeot', 'Volvo', 'Rolls Royce', 'Bentley', 'Subaru',
       'Porsche', 'LandRover', 'Geely', 'Chery', 'Smart', 'Infiniti',
       'Lynk&Co', 'Gaz', 'BYD', 'Ferrari', 'Maybach', 'Jaguar', 'Fiat',
       'Citroen'], dtype=object)

In [19]:
df_chotot["brand"] = df_chotot["brand"].str.lower()

4.year

In [20]:
df_chotot["age"].isna().sum()

np.int64(0)

In [21]:
df_chotot["age"] = 2025 - df_chotot["age"].astype(int)

In [22]:
df_chotot.sample(5)

,km,Tình_trạng,origin,body,fuel,name,price,brand,age
525,70900,Đã sử dụng,Việt Nam,Pick-up (bán tải),Dầu,RANGER XLS 2022 MẪU MỚI NEXTGEN XE CTY XHD CAO,628000000,ford,3
611,None,Mới,Nước khác,SUV / Cross over,Xăng,XForce Giảm 100 Trước Bạ Xe sẵn giao ngay,539000000,mitsubishi,0
809,96000,Đã sử dụng,Việt Nam,Sedan,Xăng,BÁN XE TOYOTA CAMRY 2016 2 0 MÀU ĐỎ ĐÔ CHÍNH CHỦ,490000000,toyota,9
1520,13000,Đã sử dụng,Việt Nam,Van,Xăng,van 5 chỗ mầu đen mới đi còn mới sx 2023,175000000,hãng khác,2
974,86000,Đã sử dụng,Mỹ,SUV / Cross over,Xăng,Mercedes Benz GLE 400 4Matic Coupe fix nhanh,1230000000,mercedes benz,9


5.status

In [23]:
df_chotot["Tình_trạng"].unique()

array(['Mới', 'Đã sử dụng'], dtype=object)

In [24]:
df_chotot = df_chotot[df_chotot["Tình_trạng"] == "Đã sử dụng"]
df_chotot.drop(columns=["Tình_trạng"], inplace=True)

6.km

In [25]:
df_chotot["km"].head(5)

17    45000
18     2800
19    80000
20    83000
21    59000
Name: km, dtype: object

In [26]:
df_chotot["km"].isna().sum()

np.int64(0)

In [27]:
df_chotot["km"].describe()

count       2332
unique       518
top       100000
freq          74
Name: km, dtype: object

In [28]:
df_chotot["km"] = df_chotot["km"].astype(int)

In [29]:
df_chotot = df_chotot[df_chotot["km"] > 100]

In [30]:
df_chotot.head()

,km,origin,body,fuel,name,price,brand,age
17,45000,Việt Nam,Sedan,Xăng,Mazda 3 2022 1 5L Premium 45000 km,555000000,mazda,3
18,2800,Việt Nam,Sedan,Xăng,Mazda 3 Sedan 2025 1 5 Luxury Đỏ 2800 km,640000000,mazda,0
19,80000,Việt Nam,Sedan,Xăng,mazda 3 2016,355000000,mazda,9
20,83000,Đức,Mui trần,Xăng,Audi A5 2010 2 0 T Quattro mui trần đẹp xuất sắc,618000000,audi,15
21,59000,Việt Nam,Sedan,Xăng,Hyundai Accent model 2024 1 4 MT BẠC SANG TRỌNG,339000000,hyundai,1


7.origin

In [31]:
df_chotot["origin"].unique()

array(['Việt Nam', 'Đức', 'Thái Lan', 'Đang cập nhật', 'Nhật Bản',
       'Hàn Quốc', 'Mỹ', 'Ấn Độ', 'Nước khác', 'Trung Quốc', 'Đài Loan'],
      dtype=object)

In [32]:
df_chotot["origin"] = df_chotot["origin"].replace({
    "Việt Nam": "trong nước"
}).apply(lambda x: "nhập khẩu" if x not in ["trong nước"] else x)

In [33]:
df_chotot["origin"].value_counts()

origin
nhập khẩu     1300
trong nước     954
Name: count, dtype: int64

8.body

In [34]:
df_chotot["body"].isna().sum()

np.int64(225)

In [35]:
df_chotot["body"].unique()

array(['Sedan', 'Mui trần', None, 'Pick-up (bán tải)', 'Hatchback',
       'SUV / Cross over', 'Van', 'Minivan (MPV)', 'Coupe (2 cửa)',
       'Kiểu dáng khác'], dtype=object)

In [36]:
df_chotot["body"] = df_chotot["body"].str.lower().str.strip()

9.fuel

In [37]:
df_chotot["fuel"].isna().sum()

np.int64(0)

In [38]:
df_chotot["fuel"].unique()

array(['Xăng', 'Dầu', 'Động cơ Hybrid', 'Điện'], dtype=object)

In [39]:
df_chotot["fuel"] = df_chotot["fuel"].apply(clean_fuel)

In [40]:
df_chotot["fuel"].unique()

array(['Xăng', 'Dầu', 'Hybrid', 'Điện'], dtype=object)

10.updata

In [41]:
df_chotot.sample(10)

,km,origin,body,fuel,name,price,brand,age
578,69000,nhập khẩu,sedan,Xăng,Bán Vios E số sàn 2022,352000000,toyota,3
1828,51000,trong nước,suv / cross over,Xăng,Tucson 2 0 Xăng Đặc Biệt 2020 Chuẩn Odo 5V,665000000,hyundai,5
698,36000,trong nước,pick-up (bán tải),Dầu,FORD RANGER XLS 4X2 AT 2023 GIA ĐÌNH LƯỚT VAY 70,615000000,ford,2
1544,60000,nhập khẩu,sedan,Xăng,Kia Cerato nhập 1 6 hatchback siêu hiếm,252000000,kia,13
1178,80000,nhập khẩu,suv / cross over,Xăng,Chính chủ Bán hoặc đổi xe Pajero Sport Máy Dầu,399000000,mitsubishi,9
1483,39360,trong nước,hatchback,Điện,VinFast VF8 ECO 2023 PIN CATL MÀU ĐEN BAO ĐẸP,730000000,vinfast,2
658,43000,nhập khẩu,suv / cross over,Xăng,Subaru Forester 2023 2 0i L phom mới full lịch sử,748000000,subaru,2
2372,83000,nhập khẩu,sedan,Xăng,ALTIS 2019 BẢN 1 8G GIÁ TỐT,475000000,toyota,6
1466,58000,trong nước,sedan,Xăng,Bán xe maybach s450,4000000000,mercedes benz,5
1118,1000,trong nước,van,Dầu,Ford Transit 18 chỗ Đen siêu lướt đi 1000km 2025,1035000000,ford,0


In [42]:
upload_to_bigquery(df_chotot, table_id=table_id, if_exists="append" )

✅ Uploaded 2254 rows to khangtestdbt.xecupredict.data_cleaned


In [43]:
#end